In [3]:
from collections import defaultdict
import uuid
import time
import heapq
import datetime


In [4]:
class Order:
    def __init__(self,instrument,order_id,side,price,quantity,timestamp):
        self.instrument = instrument
        self.order_id = order_id
        self.side = side
        self.price = price
        self.quantity = quantity
        self.timestamp = timestamp
        self.filled_quantity = 0
        
    # price/time priority
    def __lt__(self,other):
        if self.side == 'buy':
            return (self.price, -self.timestamp) > (other.price,-other.timestamp) # to get most recent buy order
        else:
            return (self.price,self.timestamp) < (other.price, other.timestamp)
           

In [5]:
class OrderBook:
    def __init__(self):
        self.bids = []
        self.asks = []
        
    def add(self,order):
        if order.side =='buy':
            heapq.heappush(self.bids,(-order.price,order.timestamp,order))
        elif order.side == 'sell':
            heapq.heappush(self.asks,(order.price,order.timestamp,order))
            
            
    def remove(self, order):
        if order.side == 'buy':
            self.bids.remove((-order.price,order.timestamp,order))
            heapq.heapify(self.bids)
        else:
            self.asks.remove((order.price,order.timestamp,order))
            heapq.heapify(self.asks)
            
    def bestBid(self):
        return self.bids[0][2] if self.bids else None

    def bestAsk(self):
        return self.asks[0][2] if self.asks else None
    
    def showBestBidAndAsk(self):
        best_bid = self.bestBid()
        best_ask = self.bestAsk()

        if (best_bid is not None and best_bid.quantity>0):
            print(f"Best Bid: Instrument: {best_bid.instrument}, Price: {best_bid.price}, Quantity: {best_bid.quantity}")
            print("----------------------------------------------------------------------------------------------------")
        else:
            print("No best bid available.")
            print("----------------------------------------------------------------------------------------------------")

        if (best_ask is not None and best_ask.quantity>0):
            print(f"Best Ask: Instrument: {best_ask.instrument}, Price: {best_ask.price}, Quantity: {best_ask.quantity}")
            print("----------------------------------------------------------------------------------------------------")
        else:
            print("No best ask available.")
            print("----------------------------------------------------------------------------------------------------")

In [6]:
class Trade:
    def __init__(self,id,name,price,volume):
        self.id = id
        self.name = name
        self.price = price
        self.volume = volume

In [7]:
class MatchedOrder:
    def __init__(self, order_id, side, price, filled_quantity, instrument, timestamp):
        self.order_id = order_id
        self.side = side
        self.price = price
        self.filled_quantity = filled_quantity
        self.instrument = instrument
        self.timestamp = timestamp

In [8]:
# class MatchedOrder:
#     def __init__(self, buy_order_id, buy_side, buy_price, buy_filled_quantity,buy_instrument,buy_timestamp):
#                  #sell_order_id, sell_side, sell_price, sell_filled_quantity,sell_instrument,sell_timestamp):
#         self.buy_order_id = buy_order_id
#         self.buy_side = buy_side
#         self.buy_price = buy_price
#         self.buy_filled_quantity = buy_filled_quantity
#         self.buy_instrument = buy_instrument
#         self.buy_timestamp = buy_timestamp
        
#         # self.sell_order_id = sell_order_id
#         # self.sell_side = sell_side
#         # self.sell_price = sell_price
#         # self.sell_filled_quantity = sell_filled_quantity
#         # self.sell_instrument = sell_instrument
#         # self.sell_timestamp = sell_timestamp

In [21]:
class MatchingEngine:
    def __init__(self):
        self.orderbooks = defaultdict(OrderBook)
        self.trades = defaultdict(list)
        self.buyerAckQueue = []
        self.sellerAckQueue = []
        self.orders = {}
        
    def placeOrder(self,instrument,side,price,quantity):
        order = Order(instrument,uuid.uuid1(),side,price,quantity,timestamp=int(time.time()))
        self.processOrder(order)
        self.orders[order.order_id] = order
        return order.order_id
    
    # Change the code to show order instrument,quantity,timestamp
    def getOrderBook(self,instrument):
        orderbook = self.orderbooks[instrument]
        # orderbook.showBestBidAndAsk()
        return orderbook.bids,orderbook.asks
    
    def showBestBidAsk(self,instrument):
        orderbook = self.orderbooks[instrument]
        orderbook.showBestBidAndAsk()
        
        
        
    
    def getTrades(self,instrument):
        return self.trades[instrument]
       
    def processOrder(self,order):
            orderbook = self.orderbooks[order.instrument]
            
            # add it toborderbook  ### TODO - remove this and add it to else condition of match
            # if (order.quantity >0):
            #     orderbook.add(order)
             
            if (order.side == 'buy'):
                print(f"Buy order {order.order_id,order.instrument,order.price,order.quantity} is added to orderbook")
                
                
            else:
                print(f"Sale order {order.order_id,order.instrument,order.price,order.quantity} is added to orderbook")
        
            if (order.side == 'buy' and orderbook.bestAsk() is not None and order.price >= orderbook.bestAsk().price):
        # Buy order crossed the spread
                self.matchBuyOrder(order, orderbook)
                # print(f"Buy order {order.order_id,order.instrument,order.price,order.quantity} matched with buy order {orderbook.bestAsk().order_id,orderbook.Ask().price,orderbook.bestAsk().quantity}")
                # print("----------------------------------------------------------------------------------------------------")
                
                
            elif (order.side == 'sell' and orderbook.bestBid() is not None and order.price <= orderbook.bestBid().price):
        # Sell order crossed the spread.
                qty = order.quantity
                self.matchSellOrder(order, orderbook)
                # print(f"Sale order {order.order_id,order.instrument,order.price,qty} matched with buy order {orderbook.bestBid().order_id,orderbook.bestBid().price,orderbook.bestBid().quantity}")
                # print("----------------------------------------------------------------------------------------------------")
            else:
                # Order did not cross the spread, place in order book
                orderbook.add(order)
                
                    
        
    def acknowledgeOrder(self, matchedOrder):
        
        timeACk = datetime.datetime.fromtimestamp(matchedOrder.timestamp)
        # Print the order details
        if (matchedOrder.side == 'buy'):
            print(f"Order acknowledgmenet for buy order {matchedOrder.order_id,matchedOrder.instrument}")
            
        else:
            print(f"Order acknowledgmenet for sale order {matchedOrder.order_id,matchedOrder.instrument}")
            
        print(f"Order ID: {matchedOrder.order_id}")
        print(f"Instrument: {matchedOrder.instrument}")
        print(f"Price: {matchedOrder.price}")
        print(f"Quantity: {matchedOrder.filled_quantity}")
        print(f"Timestamp: {timeACk}")
        print(f"Action: {'Bought' if matchedOrder.side == 'buy' else 'Sold'}")
        print("----------------------------------------------------------------------------------------------------")
        
        if matchedOrder.side == 'buy':
            self.buyerAckQueue.append(matchedOrder)
        else:
            self.sellerAckQueue.append(matchedOrder)
        
    def cancelOrder(self, order_id):
        if order_id in self.orders:
            order = self.orders[order_id]
            if order.filled_quantity == order.quantity:
                print(f"Cannot cancel order {order_id}, already fully filled")
                print("----------------------------------------------------------------------------------------------------")
                self.buyerAckQueue.append(order)
                orderbook = self.orderbooks[order.instrument]
                orderbook.remove(order)
                del self.orders[order_id]
                return True
            elif order.filled_quantity < order.quantity:
                remaining_quantity = order.quantity - order.filled_quantity
                order.quantity -= order.filled_quantity
                orderbook = self.orderbooks[order.instrument]
                orderbook.remove(order)
                del self.orders[order_id]

                print(f"Order ID: {order_id} is cancelled. Filled quantity: {order.filled_quantity}, Remaining quantity: {remaining_quantity}")
                print("----------------------------------------------------------------------------------------------------")
                return True
            else:
                return False

        else:
            print("There is no such order")
            print("----------------------------------------------------------------------------------------------------")
            return False

                
    def match(self,order):
        orderbook = self.orderbooks[order.instrument]
        
        if (order.side == 'buy' and orderbook.bestAsk() is not None and order.price >= orderbook.bestAsk().price):
    # Buy order crossed the spread.
            self.matchBuyOrder(order, orderbook)
        elif (order.side == 'sell' and orderbook.bestBid() is not None and order.price <= orderbook.bestBid().price):
    # Sell order crossed the spread.
            self.matchSellOrder(order, orderbook)
        # else:
        #     # # Order did not cross the spread, place in order book
        #     orderbook.add(order)
            
        
    def matchBuyOrder(self,order, orderbook):
        
        # if sell price is less than buy price
        if (orderbook.bestAsk().price < order.price):
            new_price = orderbook.bestAsk().price
        
        
        while orderbook.asks and order.price >= orderbook.bestAsk().price and order.quantity > 0 :
            ask = orderbook.bestAsk()
            buyMatchedOrder = []
            
            if ask.quantity <= order.quantity: # quantity for sale is less than quantity for buy
              
                
                order.quantity -= ask.quantity
                order.filled_quantity += ask.quantity
                trade = Trade(ask.order_id,ask.instrument,ask.price,ask.quantity)
                self.trades[order.instrument].append(trade)
                
                buyMatchedOrder.append(MatchedOrder(ask.order_id,'sell',ask.price,ask.quantity,ask.instrument,ask.timestamp))
                
                if new_price:
                    buyMatchedOrder.append(MatchedOrder(order.order_id,'buy',new_price,order.filled_quantity,order.instrument,order.timestamp))
                    
                else:
                     buyMatchedOrder.append(MatchedOrder(order.order_id,'buy',order.price,order.filled_quantity,order.instrument,order.timestamp))
                
               
                self.acknowledgeOrder(buyMatchedOrder[0])
                self.acknowledgeOrder(buyMatchedOrder[1])
                
                orderbook.remove(ask)
                # Acknowledgements
                
                    
                
            else:  # quantity for sale is more than quantity for buy
                trade = Trade(ask.order_id,ask.instrument,ask.price,ask.quantity)
                self.trades[order.instrument].append(trade)
            
                ask.quantity -= order.quantity
                buyMatchedOrder.append(MatchedOrder(ask.order_id,'sell',ask.price,ask.quantity,ask.instrument,ask.timestamp))
                
                if new_price:
                    buyMatchedOrder.append(MatchedOrder(order.order_id,'buy',new_price,order.quantity,order.instrument,order.timestamp))
                    
                else:
                    buyMatchedOrder.append(MatchedOrder(order.order_id,'buy',order.price,order.filled_quantity,order.instrument,order.timestamp))
                
                self.acknowledgeOrder(buyMatchedOrder[0])
                self.acknowledgeOrder(buyMatchedOrder[1])
                order.filled_quantity += order.quantity
                order.quantity = 0
                
            # if buy order not fully filled add remaining back to orderbook
            if order.quantity > 0:
                orderbook.add(order)
                
    def matchSellOrder(self, order, orderbook):
        
        ### check to see if sale price less than buy price. if it is update buy price
        if (order.price <= orderbook.bestBid().price):
            new_buy_price = order.price
        
        while orderbook.bids and order.price >= new_buy_price and order.quantity > 0:
            bid = orderbook.bestBid()
            
            sellMatchedOrder = []

            if bid.quantity <= order.quantity: # quantity for buy is less than quantity for sale
                sellMatchedOrder.append(MatchedOrder(bid.order_id,'buy',new_buy_price,bid.quantity,bid.instrument,bid.timestamp))
                sellMatchedOrder.append(MatchedOrder(order.order_id,'sell',order.price,order.quantity,order.instrument,order.timestamp)) 
                
                order.quantity -= bid.quantity
                order.filled_quantity += bid.quantity
                trade = Trade(bid.order_id,bid.instrument,new_buy_price, bid.quantity)
                self.trades[order.instrument].append(trade)
                #self.acknowledgeOrder(MatchedOrder(bid.order_id, 'buy', bid.price, bid.quantity, order.instrument, bid.timestamp))
               
                
                self.acknowledgeOrder(sellMatchedOrder[0])
                self.acknowledgeOrder(sellMatchedOrder[1])
                orderbook.remove(bid)
            else: # quantity for buy is more than quantity for sale
                
                sellMatchedOrder.append(MatchedOrder(order.order_id,'sell',new_buy_price,order.quantity,order.instrument,order.timestamp)) 
                
                trade = Trade(bid.order_id,bid.instrument,new_buy_price, bid.quantity)
                self.trades[order.instrument].append(trade)
                bid.quantity -= order.quantity
                #self.acknowledgeOrder(MatchedOrder(bid.order_id, 'buy', bid.price, order.quantity, order.instrument, bid.timestamp))
                order.filled_quantity += order.quantity
                order.quantity = 0
                sellMatchedOrder.append(MatchedOrder(bid.order_id,'buy',new_buy_price,order.filled_quantity,bid.instrument,bid.timestamp))
                
                self.acknowledgeOrder(sellMatchedOrder[0])
                self.acknowledgeOrder(sellMatchedOrder[1])

        # If the sell order is not fully filled, add the remaining part to the order book
        if order.quantity > 0:
            orderbook.add(order)
        
    

### TESTING OUTPUT (DEBUGGING)

In [10]:
# my_order = Order('ETH', 12345,'buy', 150.0, 100, 1648763456)
# print("Instrument:", my_order.instrument)
# print("Order ID:", my_order.order_id)
# print("Side:", my_order.side)
# print("Price:", my_order.price)
# print("Quantity:", my_order.quantity)
# print("Timestamp:", my_order.timestamp)
# print("Filled Quantity:", my_order.filled_quantity)

# my_order2 = Order('ETH', 145,'buy', 230.0, 100, 1648763456)
# print("Instrument:", my_order.instrument)
# print("Order ID:", my_order.order_id)
# print("Side:", my_order.side)
# print("Price:", my_order.price)
# print("Quantity:", my_order.quantity)
# print("Timestamp:", my_order.timestamp)
# print("Filled Quantity:", my_order.filled_quantity)

# myOrderBook = OrderBook()
# myOrderBook.add(my_order)
# myOrderBook.add(my_order2)
# ord = myOrderBook.bestBid()
# print(ord.order_id)

### TEST CASES

#### Place buy order n retrieve orderbook

In [22]:
engine = MatchingEngine()

# Get trades.
def getTrades(inst):
    trades = engine.getTrades(inst)
    print("Trades made: ")

    for i in trades:
        print(i.id,i.name,i.price,i.volume)

    # Get filled orders.
    if engine.buyerAckQueue:
        buy_ack = engine.buyerAckQueue.pop()
        print("Buy filled:", buy_ack.order_id, buy_ack.side,buy_ack.instrument, buy_ack.price, buy_ack.filled_quantity)
    else:
        print("No buy acknowledgements")

    if engine.sellerAckQueue:
        sell_ack = engine.sellerAckQueue.pop()
        print("Sale filled:", sell_ack.order_id, sell_ack.side, sell_ack.price, sell_ack.filled_quantity)
    else:
        print("No sell acknowledgements")


In [23]:
instrument = "ETH"

In [24]:

sell_order_id1 = engine.placeOrder("ETH", "sell", 100, 2)



Sale order (UUID('8aab09e7-e80a-11ed-a461-9078418aa6e1'), 'ETH', 100, 2) is added to orderbook


In [25]:
buy_order_id1 = engine.placeOrder("ETH", "buy", 150, 5)


Buy order (UUID('8c99ed38-e80a-11ed-b2ee-9078418aa6e1'), 'ETH', 150, 5) is added to orderbook
Order acknowledgmenet for sale order (UUID('8aab09e7-e80a-11ed-a461-9078418aa6e1'), 'ETH')
Order ID: 8aab09e7-e80a-11ed-a461-9078418aa6e1
Instrument: ETH
Price: 100
Quantity: 2
Timestamp: 2023-05-01 18:25:49
Action: Sold
----------------------------------------------------------------------------------------------------
Order acknowledgmenet for buy order (UUID('8c99ed38-e80a-11ed-b2ee-9078418aa6e1'), 'ETH')
Order ID: 8c99ed38-e80a-11ed-b2ee-9078418aa6e1
Instrument: ETH
Price: 100
Quantity: 2
Timestamp: 2023-05-01 18:25:52
Action: Bought
----------------------------------------------------------------------------------------------------


In [26]:
bids, asks = engine.getOrderBook(instrument)
print("----------------------------------------------------------------------------------------------------")
print(f"OrderBook for {instrument}")
print("Bids:", bids)
print("Asks:", asks)
print("----------------------------------------------------------------------------------------------------")

engine.showBestBidAsk("ETH")


----------------------------------------------------------------------------------------------------
OrderBook for ETH
Bids: [(-150, 1682936752, <__main__.Order object at 0x0000029BB931DD10>)]
Asks: []
----------------------------------------------------------------------------------------------------
Best Bid: Instrument: ETH, Price: 150, Quantity: 3
----------------------------------------------------------------------------------------------------
No best ask available.
----------------------------------------------------------------------------------------------------


In [27]:
getTrades(instrument)

Trades made: 
8aab09e7-e80a-11ed-a461-9078418aa6e1 ETH 100 2
Buy filled: 8c99ed38-e80a-11ed-b2ee-9078418aa6e1 buy ETH 100 2
Sale filled: 8aab09e7-e80a-11ed-a461-9078418aa6e1 sell 100 2


In [ ]:
# Create a matching engine instance.
engine = MatchingEngine()

# Place orders.
sell_order_id1 = engine.placeOrder("ETH", "sell", 100, 2)
buy_order_id1 = engine.placeOrder("ETH", "buy", 150, 5)
#buy_order_id2 = engine.placeOrder("BTC", "buy", 160, 5)

#buy_order_id3 = engine.placeOrder("BTC", "buy", 250, 3)

buy_order_id4 = engine.placeOrder("ETH", "buy", 120, 5)
buy_order_id5 = engine.placeOrder("ETH", "buy", 110, 5)





bids, asks = engine.getOrderBook(instrument)
print("----------------------------------------------------------------------------------------------------")
print(f"OrderBook for {instrument}")
print("Bids:", bids)
print("Asks:", asks)
print("----------------------------------------------------------------------------------------------------")

engine.showBestBidAsk("ETH")
